In [16]:
import os

image_folder = '/content/images'
image_count = 0

for filename in os.listdir(image_folder):
  if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
    image_count += 1

print(f"There are {image_count} images in the '{image_folder}' folder.")

There are 19618 images in the '/content/images' folder.


In [ ]:
import shutil
import os
from google.colab import files

shutil.make_archive('images', 'zip', root_dir='/content', base_dir='images')

files.download('images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
import requests
import os
import time

access_key = "srOz0e_-xrA8-htgvls9DcaOyqIMNu-xFlKmdjsDZKw"
url = "https://api.unsplash.com/search/photos"
headers = {
    "Authorization": f"Client-ID {access_key}"
}
params = {
    "query": "indian-face",
    "per_page": 30
}
if not os.path.exists('images'):
    os.makedirs('images')

page_num = 654

while True:
    params["page"] = page_num
    response = requests.get(url, headers=headers, params=params)

    time.sleep(1)

    if not data['results']:
        break

    for i, photo in enumerate(data['results']):
        img_url = photo['urls']['regular']
        img_data = requests.get(img_url).content
        with open(os.path.join('images', f'image_{page_num}_{i}.jpg'), 'wb') as handler:
            handler.write(img_data)

    print(f"Downloaded images from page {page_num}")
    page_num += 1

print('All images downloaded successfully!')

Downloaded images from page 654
Downloaded images from page 655
Downloaded images from page 656
Downloaded images from page 657
Downloaded images from page 658
Downloaded images from page 659
Downloaded images from page 660
Downloaded images from page 661
Downloaded images from page 662
Downloaded images from page 663
Downloaded images from page 664
Downloaded images from page 665
Downloaded images from page 666
Downloaded images from page 667
Downloaded images from page 668
Downloaded images from page 669
Downloaded images from page 670
Downloaded images from page 671
Downloaded images from page 672
Downloaded images from page 673
Downloaded images from page 674
Downloaded images from page 675
Downloaded images from page 676
Downloaded images from page 677
Downloaded images from page 678
Downloaded images from page 679
Downloaded images from page 680
Downloaded images from page 681
Downloaded images from page 682
Downloaded images from page 683
Downloaded images from page 684
Download

ChunkedEncodingError: ('Connection broken: IncompleteRead(64971 bytes read, 105321 more expected)', IncompleteRead(64971 bytes read, 105321 more expected))